In [1]:
'''Mounting Google Drive on the Colab notebook'''
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#file_image = '/content/gdrive/My Drive/Brain_Tumour_segmentation/Train_image.hdf5'
import h5py
#dataset has data of 484 patients. (155 images of each patient)
#data is extracted using 4 different techniques
#size of data of 1 patient is [240,240,155,4]
#for 2D segmentation we stack in 3rd dimension (axis=2)
#train_image
image_store = h5py.File("/content/gdrive/My Drive/Brain_Tumour_segmentation/Train_image.hdf5", "r")
#train_labels
label_store = h5py.File("/content/gdrive/My Drive/Brain_Tumour_segmentation/Train_label.hdf5", "r")
train_images = image_store["image"]
train_labels = label_store["label"]
#print('hi')

In [3]:
'''IMPORTING LIBRARIES'''
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
import time
import math
import os
'''Clearing tesorflow computation graph'''
tf.reset_default_graph()

In [0]:
'''DEFINING VARIABLES'''

batch_size=1              #batch size taken at a time
n_class = 4                #number of classes in the label

'''PLACEHOLDER for input and output of UNET'''
'''Here we crop the 155x240x240 image to  160x160x192 by repeating the last layer 5 times to convert 155 to 160'''
#X = tf.placeholder(shape=[None,160,160,192,4], dtype=tf.float32, name='input_image')
#y = tf.placeholder(shape=[None,160,160,192,1], dtype=tf.int64, name='hot_encode_label')
#loss_gamma = tf.placeholder(shape=[], dtype=tf.float32, name='loss_gamma')
#training = tf.placeholder_with_default(False, shape = (), name = 'training')
gm = 0.01

'''Batch variable exraction from h5py file (used by functions 'rnadom_h5py_batch' and 'test_batch')'''
out_img = np.empty((240,240,batch_size*155,4),dtype=np.float32)
out_label = np.empty((240,240,batch_size*155,1),dtype=np.int64)

In [0]:
'''COMPUTATION GRAPH Function Definitions'''


def left_filter_def(ker_size,in_chan,out_chan,name='left_filter'):
    '''Defining a filter variable to perform convolution'''
    stddev = np.sqrt(4/(ker_size*ker_size*ker_size*ker_size*in_chan*out_chan))           #HE initialization
    if stddev < 0.0008:
        stddev = 0.0008
    return (tf.Variable(tf.truncated_normal([ker_size,ker_size,ker_size,in_chan,out_chan],stddev=stddev),name=name))


def right_filter_def(ker_size,in_chan,out_chan,name='right_filter'):
    '''Defining a filter variable to perform transpose convolution'''
    stddev = np.sqrt(4/(ker_size*ker_size*ker_size*ker_size*in_chan*out_chan))
    if stddev < 0.0008:
        stddev = 0.0008
    return (tf.Variable(tf.truncated_normal([ker_size,ker_size,ker_size,out_chan,in_chan],stddev=stddev),name=name))


def Conv_layer(input_im,filter_mask,stride,activation='None',name='conv',padding="VALID"):
    '''Function to perform Convolution and apply activation filter'''
    '''Convolution'''
    conv = tf.nn.conv3d(input_im,filter_mask,strides = [1,stride,stride,stride,1], padding = padding)
    #norm_conv = tf.layers.batch_normalization(conv, training=training, momentum=0.9)
    '''Activation'''
    if activation == 'relu':
        return(tf.nn.relu(conv,name=name))
    elif activation == 'softmax':
        return(tf.nn.softmax(conv,axis=-1,name=name))
    elif activation == 'elu':
        return(tf.nn.elu(conv,name=name))
    else:
        #activation == 'None'
        return(conv)
    

def Deconv_layer(input_im,filter_mask,stride,activation='None',name='De_conv',padding="VALID"):
    '''Function to perform Transpose Convolution and apply activation filter'''
    '''Transpose Convolution'''
    inp_shape = np.shape(input_im) #tf.shape()
    out_shape = [batch_size]+[int(inp_shape[1].value*2), int(inp_shape[2].value*2),int(inp_shape[3].value*2), int(inp_shape[4].value/2)]
    
    conv = tf.nn.conv3d_transpose(input_im, filter_mask, out_shape, strides = [1,stride,stride,stride,1], padding = padding,name=name)
    #norm_conv = tf.layers.batch_normalization(conv, training=training, momentum=0.9)
    '''Activation'''
    if activation == 'relu':
        return(tf.nn.relu(conv))
    elif activation == 'softmax':
        return(tf.nn.softmax(conv,axis=-1))
    elif activation == 'elu':
        return(tf.nn.elu(conv))
    else:
        #activation == 'None'
        return(conv)




In [0]:
'''Functions for batch Extraction and pre processing'''

def normalizing_input():
    '''normalization of each input channels'''
    global out_img
    '''CHANNEL INFO'''
    # maximum value found using function called "Finding_maximum_to_normalise "
    #'''max value for dimension 4 is 5337.0'''
    #'''max value for dimension 3 is 11737.0'''
    #'''max value for dimension 2 is 9751.0'''
    #'''max value for dimension 1 is 6476.0'''
    out_img[:,:,:,0] = out_img[:,:,:,0]/6476.0
    out_img[:,:,:,1] = out_img[:,:,:,1]/9751.0
    out_img[:,:,:,2] = out_img[:,:,:,2]/11737.0
    out_img[:,:,:,3] = out_img[:,:,:,3]/5337.0
    
    

def crop_image_fit_brain(in_image):
    '''cropping size was found using the code finding_brain.ipynb'''
    left = 19
    right= 210
    top  = 38
    bot  = 199
    out_image = in_image[top:(bot-1),left:(right+1),:,:]
    return (out_image)


def Pre_processing_3D(a):
    '''Function to roll axis to convert array into[depth,width,height,channels] and the divide it in batches'''
    #print(np.shape(a))
    b = np.rollaxis(a,2, 0)
    #print(np.shape(b))
    #image shape
    out_arr = np.empty(shape=[batch_size,160,np.shape(b)[1],np.shape(b)[2],np.shape(b)[3]])
    for i in range(batch_size):
        start = i*155
        end = start+155
        out_arr[i,0:155,:,:,:] = b[start:end,:,:,:]
    
    a = [out_arr[:,154:155,:,:,:]]*5
    out_arr[:,155:160,:,:,:] = a[0]
    
    '''clippig data from front of each batch'''
    '''to fit the model we remove first 3 slices of each batch'''
    '''shape of a is [batch_size,depth,width,height,channels]'''
    out_send = out_arr[:,:,:,:,:]
    #print("arr",np.shape(out_arr),"send",np.shape(out_send))
    return (out_send)

def random_rotate(in_image,in_label):
    
    check = np.random.random(1)[0]
    if check<0.25:
        out_image = in_image[:,:,::-1,:,:]
        out_lab = in_label[:,:,::-1,:,:]
        
    elif check<0.50:
        
        out_image = in_image[:,:,:,::-1,:]
        out_lab = in_label[:,:,:,::-1,:]
        
    elif check<0.75:
        
        out_image = in_image[:,::-1,:,:,:]
        out_lab = in_label[:,::-1,:,:,:]
        
    else:
        out_image =in_image
        out_lab = in_label
    
    return (out_image,out_lab)



def random_h5py_batch(current_batch_no,permute_mat):
    '''Function to take batches randomly'''
    global out_img
    global out_label
    
    '''training info'''
    train_info = 380  #100 patients with 155 images each

    if current_batch_no == 0:
        no_of_batches = train_info//batch_size  
        permute_mat = np.random.permutation(no_of_batches)
    
    start = permute_mat[current_batch_no]*batch_size*155
    end = start + (batch_size*155)
    train_images.read_direct(out_img,np.s_[:,:,start:end,:])
    train_labels.read_direct(out_label,np.s_[:,:,start:end,:])
    current_batch_no += 1
    #print(len(out_img))
    '''Input normalization'''
    normalizing_input()
    '''normalization oof labels'''
    #out_label = out_label
    #normalizing_label()
    '''converting multi class to dual class'''
    #out_label = convert_dual_class(out_label)
    '''cropping image and labels'''
    crop_out_image = crop_image_fit_brain(out_img)
    crop_out_label = crop_image_fit_brain(out_label)
    '''Rolling axes'''
    #out_img_send = np.rollaxis(crop_out_image,2, 0)
    '''hot encoding'''
    #out_label_send = crop_out_label
    '''3D conversion'''
    out_img_send = Pre_processing_3D(crop_out_image)
    out_label_send = Pre_processing_3D(crop_out_label)
    '''Data augmentation rotation'''
    #out_img_send,out_label_send = random_rotate(out_img_send,out_label_send)
     
    last=0
    if current_batch_no == len(permute_mat):
        last=1
    
    return (out_img_send,out_label_send,current_batch_no,permute_mat,last)

def test_batch():
    '''Function to take next test batch''' 
    global out_img
    global out_label
    
    '''training and testing info'''
    train_info = 380  #100 patients with 155 images each
    test_info = 484-train_info  #100 patients with 155 images each
    
    no_of_batches = test_info//batch_size  
    permute_mat = np.random.permutation(no_of_batches)
    start = (permute_mat[0]*batch_size*155) +(train_info*155)
    end = start + (155*batch_size)
    train_images.read_direct(out_img,np.s_[:,:,start:end,:])
    train_labels.read_direct(out_label,np.s_[:,:,start:end,:])
    
    '''normalization'''
    normalizing_input()
    #normalizing_label()
    '''croping images and labels'''
    crop_out_image = crop_image_fit_brain(out_img)
    crop_out_label = crop_image_fit_brain(out_label) 
    '''3D processing'''
    out_img_send = Pre_processing_3D(crop_out_image)
    out_label_send = Pre_processing_3D(crop_out_label)
    
    return (out_img_send,out_label_send)



In [0]:

  
def hot_encode(check_image,depth=n_class,name='hot_encode'):
    '''function for hot encoding images'''
    a = tf.one_hot(indices = check_image, depth=depth,name=name,)
    b = tf.transpose(a,perm=[0,1,2,3,5,4])
    return b

#############################################################################################################################
#GENERALIZED DICE LOSS FUNCTION
#############################################################################################################################
def generalized_dice_coeff(y_true, y_pred):
    Ncl = y_pred.shape[-1]
    print(Ncl)
    w = np.zeros(shape=(Ncl,))
    w = tf.reduce_sum(y_true, axis=[0,1,2,3]) + 1
    print(np.shape(w))
    w = 1/((w**2))
    print(np.shape(w))
    # Compute gen dice coef:
    numerator = tf.reduce_sum(y_true*y_pred, axis=[0,1,2,3])
    denominator = tf.reduce_sum(y_true+y_pred, axis=[0,1,2,3])
    num=den=0
    for i in range(np.shape(w)[0]):
        num += w[i]*numerator[i]
        den += w[i]*denominator[i]
            
    #num = tf.reduce_sum(a)
    #den = tf.reduce_sum(b)
    num = num + 0.000000001
    den = den + 0.000000001
    
    gen_dice_coef = tf.identity(tf.divide((2*num),den),name='final_dice_coeff')
    
    return (gen_dice_coef)
#############################################################################################################################
#GENERALIZED FOCAL LOSS FUNCTION
#############################################################################################################################
def generalized_focal_loss(y_true, y_pred,gamma=2.0):
    
    Ncl = y_pred.shape[-1]
    w = np.zeros(shape=(Ncl,))
    w = tf.reduce_sum(y_true, axis=[0,1,2,3]) + 1
    w = 1/((w))
    y_pred = y_pred + 0.000000001                                               #to ensure that logarithm in next step doenst give math error
    
    ce = tf.multiply(y_true, -tf.log(y_pred))                                   #cross entropy (multiclass)
    fl_var = tf.multiply(y_true, tf.pow(tf.subtract(1., y_pred), gamma))        #focal loss variables (gamma*(1-pt)*(graund_truth))
    #fl = tf.multiply(alpha, tf.multiply(fl_var, ce))
    fl = tf.multiply(fl_var, ce)
    #fl=ce
    final = tf.reduce_sum(fl, axis=[0,1,2,3])
    normalized_focal = 0
    
    for i in range(np.shape(w)[0]):
        #print(i)
        #a = w[i]#/tf.reduce_sum(w)
        #b = fl[:,:,:,:,i]
        #c = (b)/(tf.reduce_max(b)+1)
        #normalized_focal += tf.reduce_sum(a*b)
        #b += w[i]*denominator[:,:,:,i]
        normalized_focal += w[i]*final[i]
    weighted_focal = tf.divide(normalized_focal,4.0,name='final_focal_loss')
    return (weighted_focal)

def generalized_MSE(y_true, y_pred):
    error = tf.square(y_true-y_pred)
    mse_half = tf.reduce_mean(error,axis=[0,1,2,3])
    mse = tf.reduce_sum(tf.divide(mse_half,4.0),name='final_focal_loss')
    return mse


#############################################################################################################################
#LOSS FUNCTION
#############################################################################################################################
def hybrid_loss(y_true, y_pred, gamma):
    dice = generalized_dice_coeff(y_true, y_pred)
    #focal = generalized_focal_loss(y_true, y_pred)
    #update = (gamma*dice) + ((1-gamma)*focal)
    focal = generalized_MSE(y_true,y_pred)
    update = (gamma*(1-dice))
    final_loss = tf.add(update,((1-gamma)*focal),name='final_loss')
    return final_loss,dice,focal

In [0]:
def predict_model1(X):
    '''Function to define the UNET model'''
    '''MODEL1 Filter definition'''
    '''LEFT'''
    
    #Combining_filter = left_filter_def(3,4,1,name='Combining_filter')

    encoder_filter1 = left_filter_def(2,4,16,name='encoder_filter1')
    encoder_filter2 = left_filter_def(2,16,32,name='encoder_filter2')
    encoder_filter3 = left_filter_def(2,32,64,name='encoder_filter3')
    encoder_filter4 = left_filter_def(2,64,128,name='encoder_filter4')
    
    one_cross_filter1 = left_filter_def(1,16,8,name='one_cross_filter1')
    one_cross_filter2 = left_filter_def(1,32,16,name='one_cross_filter2')
    one_cross_filter3 = left_filter_def(1,64,32,name='one_cross_filter3')
    one_cross_filter4 = left_filter_def(1,128,64,name='one_cross_filter4')

    down_conv1 = left_filter_def(2,8,16,name='down_conv1')
    down_conv2 = left_filter_def(2,16,32,name='down_conv2')
    down_conv3 = left_filter_def(2,32,64,name='down_conv3')

    up_conv0 = right_filter_def(3,8,4,name='up_conv1')
    up_conv1 = right_filter_def(3,16,8,name='up_conv1')
    up_conv2 = right_filter_def(3,32,16,name='up_conv2')
    up_conv3 = right_filter_def(3,64,32,name='up_conv3')

    end_conv1 = left_filter_def(3,8*2,8,name='end_conv1')
    end_conv2 = left_filter_def(3,16*3,16,name='end_conv1')
    end_conv3 = left_filter_def(3,32*3,32,name='end_conv1')
    end_conv4 = left_filter_def(3,64*2,64,name='end_conv1')

    final_filter = left_filter_def(1,4,4,name='Final_filter')
    
    
    with tf.name_scope("BACKBONE_STRUCTURE"):
        '''BLOCK1'''
        Backbone_cnn_pool1 = Conv_layer(X,encoder_filter1,stride=2,activation='relu',name='Backbone_cnn_pool1',padding='VALID')
        print('Backbone_cnn_pool1:',np.shape(Backbone_cnn_pool1))
        Backbone_cnn_pool2 = Conv_layer(Backbone_cnn_pool1,encoder_filter2,stride=2,activation='relu',name='Backbone_cnn_pool2',padding='VALID')
        print('Backbone_cnn_pool2:',np.shape(Backbone_cnn_pool2))
        Backbone_cnn_pool3 = Conv_layer(Backbone_cnn_pool2,encoder_filter3,stride=2,activation='relu',name='Backbone_cnn_pool3',padding='VALID')
        print('Backbone_cnn_pool3:',np.shape(Backbone_cnn_pool3))
        Backbone_cnn_pool4 = Conv_layer(Backbone_cnn_pool3,encoder_filter4,stride=2,activation='relu',name='Backbone_cnn_pool4',padding='VALID')
        print('Backbone_cnn_pool4:',np.shape(Backbone_cnn_pool4))

    with tf.name_scope("One_cross_one_block"):
        
        One_cross1 = Conv_layer(Backbone_cnn_pool1,one_cross_filter1,stride=1,activation='relu',name='one_cross1',padding="SAME")
        print('One_cross1:',np.shape(One_cross1))
        One_cross2 = Conv_layer(Backbone_cnn_pool2,one_cross_filter2,stride=1,activation='relu',name='one_cross2',padding="SAME")
        print('One_cross2:',np.shape(One_cross2))
        One_cross3 = Conv_layer(Backbone_cnn_pool3,one_cross_filter3,stride=1,activation='relu',name='one_cross3',padding="SAME")
        print('One_cross3:',np.shape(One_cross3))
        One_cross4 = Conv_layer(Backbone_cnn_pool4,one_cross_filter4,stride=1,activation='relu',name='one_cross4',padding="SAME")
        print('One_cross4:',np.shape(One_cross4))

    with tf.name_scope("Droping_from_one_cross"):

        drop1 = Conv_layer(One_cross1,down_conv1,stride=2,activation='relu',name='drop1',padding='VALID')
        print('drop1:',np.shape(drop1))
        drop2 = Conv_layer(One_cross2,down_conv2,stride=2,activation='relu',name='drop2',padding='VALID')
        print('drop2:',np.shape(drop2))
        drop3 = Conv_layer(One_cross3,down_conv3,stride=2,activation='relu',name='drop3',padding='VALID')
        print('drop3:',np.shape(drop3))

    with tf.name_scope("Concat_and_move_up"):

        '''Concat1'''
        concat1 = tf.concat([One_cross4,drop3],axis=4,name='CONCAT1')
        print('concat1:',np.shape(concat1))
        CNN1 = Conv_layer(concat1,end_conv4,stride=1,activation='relu',name='CNN1',padding='SAME')
        print('CNN1:',np.shape(CNN1))
        DCNN1= Deconv_layer(CNN1,up_conv3,stride=2,activation='relu',name='DE_CONV1',padding='SAME')
        print('DCNN1:',np.shape(DCNN1))

        '''Concat2'''
        concat2 = tf.concat([One_cross3,drop2,DCNN1],axis=4,name='CONCAT2')
        print('concat2:',np.shape(concat2))
        CNN2 = Conv_layer(concat2,end_conv3,stride=1,activation='relu',name='CNN2',padding='SAME')
        print('CNN2:',np.shape(CNN2))
        DCNN2= Deconv_layer(CNN2,up_conv2,stride=2,activation='relu',name='DE_CONV2',padding='SAME')
        print('DCNN2:',np.shape(DCNN2))

        '''Concat3'''
        concat3 = tf.concat([One_cross2,drop1,DCNN2],axis=4,name='CONCAT3')
        print('concat3:',np.shape(concat3))
        CNN3 = Conv_layer(concat3,end_conv2,stride=1,activation='relu',name='CNN3',padding='SAME')
        print('CNN3:',np.shape(CNN3))
        DCNN3= Deconv_layer(CNN3,up_conv1,stride=2,activation='relu',name='DE_CONV3',padding='SAME')
        print('DCNN3:',np.shape(DCNN3))

        '''Concat4'''
        concat4 = tf.concat([One_cross1,DCNN3],axis=4,name='CONCAT4')
        print('concat4:',np.shape(concat4))
        CNN4 = Conv_layer(concat4,end_conv1,stride=1,activation='relu',name='CNN4',padding='SAME')
        print('CNN4:',np.shape(CNN4))


    DCNN4= Deconv_layer(CNN4,up_conv0,stride=2,activation='relu',name='DE_CONV4',padding='SAME')
    print('DCNN4:',np.shape(DCNN4))
    final_conv = Conv_layer(DCNN4,final_filter,stride=1,activation='softmax',name='final',padding='SAME')
    print('final_conv:',np.shape(final_conv))

        
    return (final_conv)

In [0]:
def chose_train_restore(learning_rate =0.0001,n_epochs = 100):
    
    output_dir = "/content/gdrive/My Drive/Brain_Tumour_segmentation/RPG_net_main/version1_store/saving_model"
    model_checkpoint_file_base = os.path.join(output_dir, "model.ckpt")

    
    if not os.path.exists(model_checkpoint_file_base + ".meta"):
        '''FIRST TIME TRAINING'''
        print("Making new")
        brand_new = True
        
        prediction = predict_model1(X)#logits
        '''----------------------------------------------------------------------------------------------------------------------------------------------------------------'''
        with tf.name_scope("LOSS_FUNCTION"):
            '''using multi dimensional dice'''
            hot_y = hot_encode(y)
            #dice = 1+ dice_coef_multilabel(hot_y,prediction)     #dice loss for verison 1
            #dice = generalized_dice_coeff(hot_y[:,:,:,:,0], prediction)
            #focal = generalized_focal_loss(hot_y[:,:,:,:,0], prediction)
            #hybrid,dice,focal,sep = find_hybrid_loss(hot_y[:,:,:,:,:,0], prediction, gamma=0.9,alpha=0.8)
            #mean_error = ddice_coeffiff_error(hot_y[:,:,:,:,0], prediction)
            final_loss,final_dice,final_focal = hybrid_loss(hot_y[:,:,:,:,:,0], prediction,gamma=loss_gamma)
            #Ynet_loss,Ynet_dice,Ynet_focal = hybrid_loss(hot_y[:,:,:,:,:,0], Ynet_pred,gamma=0.9,Ynet_or_not=1)
        '''----------------------------------------------------------------------------------------------------------------------------------------------------------------'''
        with tf.name_scope("COST_FUNCTION"):
            '''Cost function''''''Remember to change max to min min to mx depending on loss function'''
            loss1 = tf.reduce_mean(final_loss, name="loss1")
            #loss2 = tf.reduce_mean(Ynet_loss, name="loss2")

        saver = tf.train.Saver()
        
    else:
        '''RESTORED MODEL'''
        print("Reloading existing")
        brand_new = False
        saver = tf.train.import_meta_graph(model_checkpoint_file_base + ".meta")
        g = tf.get_default_graph()
        '''Main variables'''
        #sep = g.get_tensor_by_name("LOSS_FUNCTION/mse_loss:0")
        final_dice = g.get_tensor_by_name("LOSS_FUNCTION/final_dice_coeff:0")
        final_focal = g.get_tensor_by_name("LOSS_FUNCTION/final_focal_loss:0")
        #hybrid = g.get_tensor_by_name("LOSS_FUNCTION/hybrid_loss:0")
        final_loss = g.get_tensor_by_name("LOSS_FUNCTION/final_loss:0")
        prediction = g.get_tensor_by_name("final:0") 
        loss1 = g.get_tensor_by_name("COST_FUNCTION/loss1:0")
        '''ynet variables'''
        #Ynet_dice = g.get_tensor_by_name("LOSS_FUNCTION/Ynet_dice_coeff:0")
        #Ynet_focal = g.get_tensor_by_name("LOSS_FUNCTION/Ynet_focal_loss:0")
        #hybrid = g.get_tensor_by_name("LOSS_FUNCTION/hybrid_loss:0")
        #Ynet_loss = g.get_tensor_by_name("LOSS_FUNCTION/Ynet_loss:0")
        #Ynet_pred = g.get_tensor_by_name("YNET/Softmax:0") 
        #loss2 = g.get_tensor_by_name("COST_FUNCTION/loss2:0")
        
        X = g.get_tensor_by_name("input_image:0")
        y = g.get_tensor_by_name("hot_encode_label:0")
        loss_gamma = g.get_tensor_by_name("loss_gamma:0")


    
    
    
    '''TRAINING'''
    '''starting session'''
    gpu_option = tf.GPUOptions(per_process_gpu_memory_fraction=0.5)
    with tf.Session(config=tf.ConfigProto(gpu_options=gpu_option)) as sess:
        '''Initializing optimizer'''
        if brand_new:
            optimizer1 = tf.train.AdamOptimizer(learning_rate).minimize(loss1)
            #optimizer2 = tf.train.AdamOptimizer(learning_rate).minimize(loss2)
            init = tf.global_variables_initializer()
            sess.run(init)
            tf.add_to_collection("optimizer1", optimizer1)
            #tf.add_to_collection("optimizer2", optimizer2)
        else:
            saver = tf.train.Saver()
            saver.restore(sess, model_checkpoint_file_base)
            optimizer1 = tf.get_collection("optimizer1")[0]
            #optimizer2 = tf.get_collection("optimizer2")[0]
        
        for epoch in range(12,n_epochs):
            current_batch_no = 0
            permute_mat = 0
            iteration = 0
            while(1):
                #with tf.device('/cpu:0'):
                epoch_x,epoch_y,current_batch_no,permute_mat,last = random_h5py_batch(current_batch_no,permute_mat)
                sess_result1 = sess.run(optimizer1, feed_dict={X: epoch_x, y: epoch_y, loss_gamma: gm})
                #print ("epoch",epoch+1,"batch",iteration+1)#,"Cost",sess_results[0])
                
                '''DICE Coefficient for iteration'''
                #with tf.device('/cpu:0'):
                if iteration%10==0:
                    #acc_train = 1-(dice.eval(feed_dict={X: epoch_x, y: epoch_y}))
                    final_train_loss,final_train_dice,final_train_focal = sess.run([final_loss,final_dice,final_focal], feed_dict={X: epoch_x, y: epoch_y, loss_gamma: gm})
                    test_images, test_labels = test_batch()
                    #acc_test = 1-(dice.eval(feed_dict={X: test_images, y: test_labels}))
                    final_test_loss,final_test_dice,final_test_focal = sess.run([final_loss,final_dice,final_focal], feed_dict={X: test_images, y: test_labels, loss_gamma: gm})
                    #print("------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
                    print("MAIN: Minibatch at","Epoch", epoch+1,"batch",iteration+1, "Train Loss:", final_train_loss, "Train Dice Coeff",final_train_dice,"Train Focal Loss",final_train_focal,"Test Loss:", final_test_loss, "Test Dice Coeff",final_test_dice,"Test Focal Loss",final_test_focal)
                    #print("YNET: Minibatch at","Epoch", epoch+1,"batch",iteration+1, "Train Loss:", Ynet_train_loss, "Train Dice Coeff",Ynet_train_dice,"Train Focal Loss",Ynet_train_focal,"Test Loss:", Ynet_test_loss, "Test Dice Coeff",Ynet_test_dice,"Test Focal Loss",Ynet_test_focal)
                    #print("------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
                    #print("After Epoch", epoch+1, "Hybrid Train accuracy:", 1-hybrid_train, "Dice Train accuracy:", 1-dice_train, "Focal Train accuracy:", 1-focal_train, "MSE Train accuracy:", 1-diff_train)
                    '''saving model after every 10 iterations'''
                    save_path = tf.train.Saver(max_to_keep=1).save(sess, model_checkpoint_file_base)
                if last ==1:
                    break
                iteration +=1
            test_images, test_labels = test_batch()
            #hybrid_test,dice_test,focal_test,diff_test = sess.run([hybrid,dice,focal,sep], feed_dict={X: test_images, y: test_labels})
            #diff_test = sess.run(dice, feed_dict={X: test_images, y: test_labels})
            final_test_loss,final_test_dice,final_test_focal = sess.run([final_loss,final_dice,final_focal], feed_dict={X: test_images, y: test_labels, loss_gamma: gm})
            print("#################################################################################################################################")
            #print("After Epoch", epoch+1, "Hybrid Test accuracy:", 1-hybrid_test, "Dice Test accuracy:", 1-dice_test, "Focal Test accuracy:", 1-focal_test, "MSE Test accuracy:", 1-diff_test)
            print("MAIN: After Epoch", epoch+1,"Test Loss:", final_test_loss, "Test Dice Coeff",final_test_dice,"Test Focal Loss",final_test_focal)
            #print("YNET: After Epoch", epoch+1,"Test Loss:", Ynet_test_loss, "Test Dice Coeff",Ynet_test_dice,"Test Focal Loss",Ynet_test_focal)
            print("#################################################################################################################################")
            '''saving model after each epoch'''
            save_path = tf.train.Saver(max_to_keep=1).save(sess, model_checkpoint_file_base)
            
            if epoch % 1 == 0:
                test_example =   test_images
                test_example_gt = test_labels#np.rollaxis(test_labels,2,0)
                sess_results = sess.run(prediction,feed_dict={X:test_example})

                sess_results = sess_results[0,100,:,:,1] + (2*sess_results[0,100,:,:,2]) + (3*sess_results[0,100,:,:,3])
                test_example = test_example[0,100,:,:,3]
                test_example_gt = test_example_gt[0,100,:,:,:]
                
                plt.figure()
                plt.imshow(np.squeeze(test_example),cmap='gray')
                plt.axis('off')
                plt.title('Original Image')
                plt.savefig('/content/gdrive/My Drive/Brain_Tumour_segmentation/RPG_net_main/version1_store/result/'+str(epoch)+"a_Original_Image.png")
                 
                plt.figure()
                plt.imshow(np.squeeze(test_example_gt),cmap='gray')
                plt.axis('off')
                plt.title('Ground Truth Mask')
                plt.savefig('/content/gdrive/My Drive/Brain_Tumour_segmentation/RPG_net_main/version1_store/result/'+str(epoch)+"b_Original_Mask.png")

                plt.figure()
                plt.imshow(np.squeeze(sess_results),cmap='gray')
                plt.axis('off')
                plt.title('Generated Mask')
                plt.savefig('/content/gdrive/My Drive/Brain_Tumour_segmentation/RPG_net_main/version1_store/result/'+str(epoch)+"c_Generated_Mask.png")

                plt.close('all')

    

In [0]:
chose_train_restore()

Reloading existing
INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/Brain_Tumour_segmentation/RPG_net_main/version1_store/saving_model/model.ckpt
MAIN: Minibatch at Epoch 13 batch 1 Train Loss: 0.01939648 Train Dice Coeff 0.0014935876 Train Focal Loss 0.009506481 Test Loss: 0.028032174 Test Dice Coeff 0.0015279648 Test Focal Loss 0.01822975
MAIN: Minibatch at Epoch 13 batch 11 Train Loss: 0.019132014 Train Dice Coeff 0.0031647296 Train Focal Loss 0.009256224 Test Loss: 0.019713068 Test Dice Coeff 0.0011830204 Test Focal Loss 0.00982313
MAIN: Minibatch at Epoch 13 batch 21 Train Loss: 0.016259726 Train Dice Coeff 0.001061222 Train Focal Loss 0.0063336743 Test Loss: 0.024616022 Test Dice Coeff 0.0053913933 Test Focal Loss 0.014818118
MAIN: Minibatch at Epoch 13 batch 31 Train Loss: 0.019284979 Train Dice Coeff 0.0025942116 Train Focal Loss 0.00940497 Test Loss: 0.033407792 Test Dice Coeff 0.00532449 Test Focal Loss 0.023698017
MAIN: Minibatch at Epoch 13 batch 41 Train 

In [0]:
print(np.shape(test_example))